In [1]:
import hashlib
import json
from time import time
import os

In [2]:
class Block:
    def __init__(self, index, timestamp, data, previous_hash):
        self.index = index
        self.timestamp = timestamp
        self.data = data
        self.previous_hash = previous_hash
        self.hash = self.calculate_hash()
    
    def calculate_hash(self):
        block_string = json.dumps({
            "index": self.index,
            "timestamp": self.timestamp,
            "data": self.data,
            "previous_hash": self.previous_hash
        }, sort_keys=True).encode()
        
        return hashlib.sha256(block_string).hexdigest()
    
    def to_dict(self):
        return {
            "index": self.index,
            "timestamp": self.timestamp,
            "data": self.data,
            "previous_hash": self.previous_hash,
            "hash": self.hash
        }


In [3]:
class Blockchain:
    def __init__(self, filename="../blockchain/data/blockchain.json"):
        self.filename = filename
        self.chain = []
        self.load_chain()
        if not self.chain:
            self.chain.append(self.create_genesis_block())
            self.save_chain()
    
    def create_genesis_block(self):
        return Block(0, time(), "Genesis Block", "0")
    
    def get_last_block(self):
        return self.chain[-1]
    
    def add_block(self, data):
        last_block = self.get_last_block()
        new_block = Block(
            index=last_block.index + 1,
            timestamp=time(),
            data=data,
previous_hash=last_block.hash
        )
        self.chain.append(new_block)
        self.save_chain()
    
    def is_chain_valid(self):
        for i in range(1, len(self.chain)):
            current = self.chain[i]
            previous = self.chain[i-1]
            
            if current.hash != current.calculate_hash():
                print(f"Tampered block at index {current.index}")
                return False
            if current.previous_hash != previous.hash:
                print(f"Previous hash mismatch at block {current.index}")
                return False
        return True
    
    def save_chain(self):
        with open(self.filename, "w") as f:
            json.dump([block.to_dict() for block in self.chain], f, indent=4)
    
    def load_chain(self):
        if os.path.exists(self.filename):
            with open(self.filename, "r") as f:
                loaded_blocks = json.load(f)
                self.chain = []
                for blk in loaded_blocks:
                    block = Block(
                        index=blk["index"],
                        timestamp=blk["timestamp"],
                        data=blk["data"],
                        previous_hash=blk["previous_hash"]
                    )
                    self.chain.append(block)


In [4]:
# Initialize blockchain
audit_chain = Blockchain()

# Add audit logs with your chosen names
audit_chain.add_block({"user": "Pen", "action": "Login"})
audit_chain.add_block({"user": "Wayne", "action": "Update Record"})
audit_chain.add_block({"user": "Keith", "action": "Delete Record"})


In [5]:
print("=== Blockchain ===")
for block in audit_chain.chain:
    print(f"Index: {block.index}, Data: {block.data}, Hash: {block.hash}")


=== Blockchain ===
Index: 0, Data: Genesis Block, Hash: 0d0ae940c245791cabece713365b9c694ea056f42a106f174a0803cd1766e7ba
Index: 1, Data: {'user': 'Pen', 'action': 'Login'}, Hash: 63fcdb84b98d35cf6330773b66cbf51743e715fc499fa8934ff1caa63b979deb
Index: 2, Data: {'user': 'Wayne', 'action': 'Update Record'}, Hash: c79d9c19c662ad277216c764c0ce6e266f4b990f9c369bc0d0a61934b87c98f5
Index: 3, Data: {'user': 'Keith', 'action': 'Delete Record'}, Hash: 1414459b3093fa0ec9e808e95e6eeb0177cd2056ea67184a0f54902184b3b43d


In [6]:
print("\nBlockchain valid?", audit_chain.is_chain_valid())



Blockchain valid? True
